<a href="https://colab.research.google.com/github/190905186/Automation-scripts/blob/main/LayerZeroAirdrop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!wget https://layerzerodataset.s3.us-east-2.amazonaws.com/2024-05-15-snapshot1_transactions.csv.gz

--2024-05-18 06:51:38--  https://layerzerodataset.s3.us-east-2.amazonaws.com/2024-05-15-snapshot1_transactions.csv.gz
Resolving layerzerodataset.s3.us-east-2.amazonaws.com (layerzerodataset.s3.us-east-2.amazonaws.com)... 3.5.130.190, 52.219.107.26, 3.5.131.39, ...
Connecting to layerzerodataset.s3.us-east-2.amazonaws.com (layerzerodataset.s3.us-east-2.amazonaws.com)|3.5.130.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15782276060 (15G) [application/x-gzip]
Saving to: ‘2024-05-15-snapshot1_transactions.csv.gz.1’

2024-05-15-snapshot 100%[===================>]  14.70G  50.2MB/s    in 5m 0s   

2024-05-18 06:56:38 (50.2 MB/s) - ‘2024-05-15-snapshot1_transactions.csv.gz.1’ saved [15782276060/15782276060]



In [4]:
!apt update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,118 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 2,351 kB in 2s (1,211 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
45 packages can be 

In [5]:
!pip install pyspark findspark
import findspark
findspark.init()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=b3907610be442c3aaf98d8eaf989db595608eef8936df545c7b883d1c37613e3
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
  .master('local[*]') \
  .appName('YourAppName') \
  .getOrCreate()


In [14]:
import pyspark.sql.functions as F

from pyspark.sql.types import StructType, StringType, TimestampType, DoubleType,StructField

schema = StructType([
    StructField("SOURCE_CHAIN", StringType(), True),
    StructField("SOURCE_TRANSACTION_HASH", StringType(), True),
    StructField("SOURCE_CONTRACT", StringType(), True),
    StructField("DESTINATION_CHAIN", StringType(), True),
    StructField("DESTINATION_TRANSACTION_HASH", StringType(), True),
    StructField("DESTINATION_CONTRACT", StringType(), True),
    StructField("SENDER_WALLET", StringType(), True),
    StructField("SOURCE_TIMESTAMP_UTC", TimestampType(), True),
    StructField("PROJECT", StringType(), True),
    StructField("NATIVE_DROP_USD", DoubleType(), True),  # Assuming USD values are numerical
    StructField("STARGATE_SWAP_USD", DoubleType(), True)  # Assuming USD values are numerical
])

# Assuming S3 filesystem is mounted (refer to previous responses for mounting instructions)
df = spark.read.csv(
    "2024-05-15-snapshot1_transactions.csv.gz",
    schema=schema,
    header=True
    # Infer schema if you don't know it beforehand
)

In [15]:
df.printSchema()

root
 |-- SOURCE_CHAIN: string (nullable = true)
 |-- SOURCE_TRANSACTION_HASH: string (nullable = true)
 |-- SOURCE_CONTRACT: string (nullable = true)
 |-- DESTINATION_CHAIN: string (nullable = true)
 |-- DESTINATION_TRANSACTION_HASH: string (nullable = true)
 |-- DESTINATION_CONTRACT: string (nullable = true)
 |-- SENDER_WALLET: string (nullable = true)
 |-- SOURCE_TIMESTAMP_UTC: timestamp (nullable = true)
 |-- PROJECT: string (nullable = true)
 |-- NATIVE_DROP_USD: double (nullable = true)
 |-- STARGATE_SWAP_USD: double (nullable = true)



In [16]:
df.show(5,truncate=False)

+----------------------+------------------------------------------------------------------+------------------------------------------+-----------------+------------------------------------------------------------------+------------------------------------------+------------------------------------------+--------------------+-------+---------------+-----------------+
|SOURCE_CHAIN          |SOURCE_TRANSACTION_HASH                                           |SOURCE_CONTRACT                           |DESTINATION_CHAIN|DESTINATION_TRANSACTION_HASH                                      |DESTINATION_CONTRACT                      |SENDER_WALLET                             |SOURCE_TIMESTAMP_UTC|PROJECT|NATIVE_DROP_USD|STARGATE_SWAP_USD|
+----------------------+------------------------------------------------------------------+------------------------------------------+-----------------+------------------------------------------------------------------+------------------------------------------+

In [36]:
from pyspark.sql.functions import to_date, year, month, dayofmonth, hour, minute, second, concat, dayofweek,lit,format_string,date_format,col

df = df.withColumn("date", to_date("SOURCE_TIMESTAMP_UTC"))
df = df.withColumn("year", year(to_date("SOURCE_TIMESTAMP_UTC")))
df = df.withColumn("month", month(to_date("SOURCE_TIMESTAMP_UTC")))
df = df.withColumn("day", dayofmonth(to_date("SOURCE_TIMESTAMP_UTC")))
df = df.withColumn("hour",hour("SOURCE_TIMESTAMP_UTC"))
df = df.withColumn("minute",minute("SOURCE_TIMESTAMP_UTC"))
df = df.withColumn("second",second("SOURCE_TIMESTAMP_UTC"))
df = df.withColumn("time", concat(hour("SOURCE_TIMESTAMP_UTC"), lit(":"), minute("SOURCE_TIMESTAMP_UTC"), lit(":"), second("SOURCE_TIMESTAMP_UTC")))
df = df.withColumn("yyyymm", date_format(to_date("SOURCE_TIMESTAMP_UTC"), "yyyyMM"))
df = df.withColumn("yyyymm",col("yyyymm").cast('int'))

In [37]:
df.printSchema()

root
 |-- SOURCE_CHAIN: string (nullable = true)
 |-- SOURCE_TRANSACTION_HASH: string (nullable = true)
 |-- SOURCE_CONTRACT: string (nullable = true)
 |-- DESTINATION_CHAIN: string (nullable = true)
 |-- DESTINATION_TRANSACTION_HASH: string (nullable = true)
 |-- DESTINATION_CONTRACT: string (nullable = true)
 |-- SENDER_WALLET: string (nullable = true)
 |-- SOURCE_TIMESTAMP_UTC: timestamp (nullable = true)
 |-- PROJECT: string (nullable = true)
 |-- NATIVE_DROP_USD: double (nullable = true)
 |-- STARGATE_SWAP_USD: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- time: string (nullable = true)
 |-- yyyymm: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- second: integer (nullable = true)



In [38]:
df.show(5,truncate=False)

+----------------------+------------------------------------------------------------------+------------------------------------------+-----------------+------------------------------------------------------------------+------------------------------------------+------------------------------------------+--------------------+-------+---------------+-----------------+----+-----+---+-------+------+----------+----+------+------+
|SOURCE_CHAIN          |SOURCE_TRANSACTION_HASH                                           |SOURCE_CONTRACT                           |DESTINATION_CHAIN|DESTINATION_TRANSACTION_HASH                                      |DESTINATION_CONTRACT                      |SENDER_WALLET                             |SOURCE_TIMESTAMP_UTC|PROJECT|NATIVE_DROP_USD|STARGATE_SWAP_USD|year|month|day|time   |yyyymm|date      |hour|minute|second|
+----------------------+------------------------------------------------------------------+------------------------------------------+--------